### Code Ingession

In [27]:
from langchain_core.documents import Document
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Tuple, Any
from sklearn.metrics.pairwise import cosine_similarity
from typing import List
import os


doc = Document(
    page_content="Hello I am creating a document to create a RAG",
    metadata = {
        "pages" : 1,
        "source" : "txt file",
        "date" : "2025-09-05",
        "author" : "Vishal"
    }
    
)

doc

Document(metadata={'pages': 1, 'source': 'txt file', 'date': '2025-09-05', 'author': 'Vishal'}, page_content='Hello I am creating a document to create a RAG')

In [28]:
## Creata a simple file

import os
os.makedirs("../data/text_files", exist_ok=True)

In [29]:
sample_texts={
    "../data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "../data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """

}

for filepath,content in sample_texts.items():
    with open(filepath,'w',encoding="utf-8") as f:
        f.write(content)

print("✅ Sample text files created!")

✅ Sample text files created!


In [30]:
from langchain.document_loaders import TextLoader

from langchain_community.document_loaders import TextLoader

loader = TextLoader("../data/text_files/python_intro.txt", encoding="utf-8")
doc = loader.load()
print( doc)

[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]


In [31]:
from langchain_community.document_loaders import DirectoryLoader

## load the directoy of txt file
dir_loader = DirectoryLoader(
    "../data/text_files", 
    glob="**/*.txt", ## Pattern to match
    loader_cls=TextLoader, ## Loader class to use
    loader_kwargs={'encoding':'utf-8'},
    show_progress=False
)

documents = dir_loader.load()
documents

[Document(metadata={'source': '..\\data\\text_files\\machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data\n3. Reinforcement Learning: Learning through rewards and penalties\n\nApplications include image recognition, speech processing, and recommendation systems\n\n\n    '),
 Document(metadata={'source': '..\\data\\text_files\\python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popu

In [32]:
from langchain_community.document_loaders import PyMuPDFLoader, PyPDFLoader
from pathlib import Path

## load the pdf files

### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")
all_pdf_documents

Found 2 PDF files to process

Processing: The Compound Effect - Darren hardy - PDF Room.pdf
  ✓ Loaded 195 pages

Processing: The Power of Habit_ Why We Do What We Do in Life and Business ( PDFDrive ).pdf
  ✓ Loaded 295 pages

Total documents loaded: 490


[Document(metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20180621132923', 'source': '..\\data\\pdf\\The Compound Effect - Darren hardy - PDF Room.pdf', 'total_pages': 195, 'page': 0, 'page_label': '1', 'source_file': 'The Compound Effect - Darren hardy - PDF Room.pdf', 'file_type': 'pdf'}, page_content=''),
 Document(metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20180621132923', 'source': '..\\data\\pdf\\The Compound Effect - Darren hardy - PDF Room.pdf', 'total_pages': 195, 'page': 1, 'page_label': '2', 'source_file': 'The Compound Effect - Darren hardy - PDF Room.pdf', 'file_type': 'pdf'}, page_content='About the Author \nThe chair once occupied by Orison Swett Marden, W. \nClement Stone, Napoleon Hill, and Og Mandino is where \nSUCCESS publisher and editorial director Darren Hardy now \nsits. Darren has been a leader in the personal-development \nindustry for sixteen years, having led two personal-\ndevelopment-based television netw

In [33]:
### Chunking 
### Text splitting get into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [34]:
chunks=split_documents(all_pdf_documents)
chunks

Split 490 documents into 1423 chunks

Example chunk:
Content: About the Author 
The chair once occupied by Orison Swett Marden, W. 
Clement Stone, Napoleon Hill, and Og Mandino is where 
SUCCESS publisher and editorial director Darren Hardy now 
sits. Darren has...
Metadata: {'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20180621132923', 'source': '..\\data\\pdf\\The Compound Effect - Darren hardy - PDF Room.pdf', 'total_pages': 195, 'page': 1, 'page_label': '2', 'source_file': 'The Compound Effect - Darren hardy - PDF Room.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20180621132923', 'source': '..\\data\\pdf\\The Compound Effect - Darren hardy - PDF Room.pdf', 'total_pages': 195, 'page': 1, 'page_label': '2', 'source_file': 'The Compound Effect - Darren hardy - PDF Room.pdf', 'file_type': 'pdf'}, page_content='About the Author \nThe chair once occupied by Orison Swett Marden, W. \nClement Stone, Napoleon Hill, and Og Mandino is where \nSUCCESS publisher and editorial director Darren Hardy now \nsits. Darren has been a leader in the personal-development \nindustry for sixteen years, having led two personal-\ndevelopment-based television networks—The People’s \nNetwork (TPN), and The Success Training Network (TSTN)—\nproducing and launching more \nthan a thousand TV shows, \nlive events, and products \nand programs with many of \nthe world’s top experts. \nDarren is a product of \nthe principles he reveals in \nThe Compound Effect. As an \nentrepreneur, Darren was \near

In [35]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Tuple, Any
from sklearn.metrics.pairwise import cosine_similarity
from typing import List
import os


In [36]:
### Embedding & VectorDBStore


class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


In [37]:
### Vector Store

In [38]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 992


In [39]:
### Convert the text to embeddings
texts = [doc.page_content for doc in chunks]

### Generate the embeddings
embddings = embedding_manager.generate_embeddings(texts)

###Store it into the vector database
vectorstore.add_documents(chunks,embddings)


Generating embeddings for 1423 texts...


Batches: 100%|██████████| 45/45 [01:03<00:00,  1.42s/it]


Generated embeddings with shape: (1423, 384)
Adding 1423 documents to vector store...
Successfully added 1423 documents to vector store
Total documents in collection: 2415


In [40]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

rag_retriever

In [51]:
rag_retriever.retrieve("Compound")

Retrieving documents for query: 'Compound'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 86.14it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)


[]